In [12]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from flask import Flask, render_template, redirect

import requests
import pymongo
import pandas as pd

In [13]:
executable_path = {'executable_path': '../chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [14]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [21]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[1].text
print(news_title)

NASA's Perseverance Rover Gives High-Definition Panoramic View of Landing Site


In [24]:
news_parg=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_parg)

A 360-degree panorama taken by the rover’s Mastcam-Z instrument will be discussed during a public video chat this Thursday.


### JPL Mars Space Images - Featured Image

In [28]:
jpl_url="https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [42]:
img_url=soup.find_all('img', class_='headerimage')[0]['src']
featured_image_url=jpl_url+img_url
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars2.jpg'

### Mars Facts